## 必要なライブラリのimport

In [1]:
!pip install numerapi
!pip install lightgbm

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import numerapi
import xgboost as xgb
from sklearn.metrics import log_loss

In [3]:
# predictions.csv : id / prediction の2項目のみ

## numerapiを使ってデータセットのダウンロード

In [4]:
# numerapiを使えばデータセットのダウンロードが簡単にできる
#インスタンス化（numerapiを使うための準備）
napi = numerapi.NumerAPI(verbosity="info")

# 現在のラウンドのデータセットをダウンロードして解凍する。
napi.download_current_dataset(unzip=True)

2021-10-09 18:00:31,983 INFO numerapi.utils: starting download
./numerai_dataset_285.zip: 425MB [00:26, 16.3MB/s]                           
2021-10-09 18:00:58,016 INFO numerapi.base_api: unzipping file...


'./numerai_dataset_285.zip'

## 準備：トーナメントの現在のラウンド数を取得

In [5]:
# numerai_dataset_321/numerai_training_data.csv でトレーニングデータのファイル名
# numerai_dataset_321/numerai_tournament_data.csv でトーナメントデータのファイル名

# まずは現在のトーナメントのラウンド数を取得(int型)
current_ds = napi.get_current_round()
print(current_ds)

# ここはnumerai_dataset_321のようなパスを得るため
latest_round = os.path.join('numerai_dataset_'+str(current_ds))
print(latest_round)

285
numerai_dataset_285


## トレーニングデータとトーナメントデータの読み込み

In [6]:
print("# データの読み込み中...")
# トレーニングデータをCSVから読み込む。　set_indexでどの列をindexにするか？を決める
# train_xは学習データ、train_yは目的変数、test_xはテストデータ
# pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）
train = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
train_x = train.drop(['era', 'data_type', 'target'], axis=1)
train_y = train['target']

# データの読み込み中...


In [7]:
# トーナメントデータをCSVから読み込む。
test = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
test_x = test.drop(['era', 'data_type', 'target'], axis=1)

feature_names = [f for f in train_x.columns if "feature" in f]

In [8]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

## データの確認

In [9]:
train_x.head()
train_x.shape

(501808, 310)

In [10]:
test_x.tail()
test_x

,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,feature_charisma27,feature_charisma28,...,feature_wisdom7,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,0.25,0.75,0.50,0.50,0.00,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,0.5,0.25,...,0.00,0.00,0.00,0.25,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00
n000920ed083903f,0.75,0.50,0.75,1.00,0.50,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,0.0,0.75,...,0.50,0.50,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50
n0038e640522c4a6,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,0.5,0.50,...,0.00,0.25,0.50,0.00,0.00,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00
n004ac94a87dc54b,0.75,1.00,1.00,0.50,0.00,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,0.0,0.75,...,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25
n0052fe97ea0c05f,0.25,0.50,0.50,0.25,1.00,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,0.5,0.00,...,1.00,0.00,0.50,0.50,0.00,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.00,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffe489171b01328

In [11]:
feature_names
len(feature_names)

310

## モデルのトレーニング

In [12]:
# 特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(test_x)

# ハイパーパラメータの設定
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
num_round = 50

# 学習の実行
# バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
# watchlistには学習データおよびバリデーションデータをセットする
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist)

# バリデーションデータでのスコアの確認
va_pred = model.predict(dvalid)
# score = log_loss(va_y, va_pred)
# print(f'logloss: {score:.4f}')

[0]	train-error:0.490815	eval-error:0.496114
[1]	train-error:0.487675	eval-error:0.494795
[2]	train-error:0.485749	eval-error:0.493934
[3]	train-error:0.484222	eval-error:0.493918
[4]	train-error:0.482655	eval-error:0.493842
[5]	train-error:0.481587	eval-error:0.494057
[6]	train-error:0.480801	eval-error:0.493252
[7]	train-error:0.47968	eval-error:0.493236
[8]	train-error:0.47883	eval-error:0.493492
[9]	train-error:0.478171	eval-error:0.493049
[10]	train-error:0.477228	eval-error:0.493037
[11]	train-error:0.476431	eval-error:0.492455
[12]	train-error:0.475459	eval-error:0.492411
[13]	train-error:0.47466	eval-error:0.492304
[14]	train-error:0.474079	eval-error:0.491778
[15]	train-error:0.473371	eval-error:0.492232
[16]	train-error:0.472485	eval-error:0.492089
[17]	train-error:0.471795	eval-error:0.492109
[18]	train-error:0.47132	eval-error:0.492499
[19]	train-error:0.470471	eval-error:0.492567
[20]	train-error:0.469898	eval-error:0.492782
[21]	train-error:0.469235	eval-error:0.492957
[2

## 予測をする

In [13]:
# 予測（二値の予測値ではなく、1である確率を出力するようにしている）
pred = model.predict(dtest)

In [14]:
# モニタリングをloglossで行い、アーリーストッピングの観察するroundを20とする
params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71,
          'eval_metric': 'logloss'}
num_round = 500
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist,
                  early_stopping_rounds=20)

# 最適な決定木の本数で予測を行う
pred = model.predict(dtest, ntree_limit=model.best_ntree_limit)

[0]	train-logloss:0.693069	eval-logloss:0.69316
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.692906	eval-logloss:0.693119
[2]	train-logloss:0.692764	eval-logloss:0.693086
[3]	train-logloss:0.692634	eval-logloss:0.693065
[4]	train-logloss:0.692505	eval-logloss:0.693059
[5]	train-logloss:0.692386	eval-logloss:0.693063
[6]	train-logloss:0.692271	eval-logloss:0.693043
[7]	train-logloss:0.692161	eval-logloss:0.693035
[8]	train-logloss:0.692052	eval-logloss:0.693035
[9]	train-logloss:0.691949	eval-logloss:0.693022
[10]	train-logloss:0.69184	eval-logloss:0.693035
[11]	train-logloss:0.691742	eval-logloss:0.693033
[12]	train-logloss:0.691641	eval-logloss:0.693035
[13]	train-logloss:0.691544	eval-logloss:0.693037
[14]	train-logloss:0.691447	eval-logloss:0.69305
[15]	train-logloss:0.691352	eval-logloss:0.693047
[16]	train-logloss:0.691257	eval-logloss:0.693056
[17]	train-log

In [15]:
pred

array([0.48011816, 0.49132678, 0.5209861 , ..., 0.5449368 , 0.4882818 ,
       0.5131371 ], dtype=float32)

In [16]:
# predictions = model.predict(tournament_data[feature_names])

In [17]:
# 予測結果をデータフレームの予測列とした。
# tournament_data['prediction'] = predictions
test['prediction'] = pred

In [18]:
test

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target,prediction
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.00,0.75,0.50,0.25,0.50,0.50,0.25,0.00,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.00,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.25,0.50,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.00,0.00,0.00,0.25,0.480118
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.50,0.00,0.00,0.75,0.25,0.00,0.75,0.50,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.00,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.25,1.00,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.50,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.50,0.50,0.50,0.50,0.50,0.491327
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.50,1.00,1.00,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.50,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.50,0.00,0.00,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.50,0.50,0.00,1.00,0.520986
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.00,0.00,0.00,0.50,0.75,1.00,0.75,0.00,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.00,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.00,0.00,0.25,0.25,0.50,0.517691
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.00,0.50,0.50,0.25,0.25,0.50,0.50,1.00,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.00,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.50,0.50,0.00,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.00,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.00,0.00,0.25,1.00,0.75,0.507984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [19]:
# トーナメント名
TOURNAMENT_NAME = "nomi"

In [20]:
# tournament_data['prediction'].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")
test['prediction'].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")

## 予測をAPIキーを使って提出

In [21]:
# APIキーの設定
public_id = "F2MPPY2RMC3MXVG44KBGRRCTZOP42ETW"
secret_key = "HDPZ7DMD33UOOQHPOTNQMOIEM75P63F75OYUOJP7G4K4EZRQX7E5PS6AS34E4GCW"
model_id = "b1a80b46-5cfc-4a0c-97a8-b050f1973242"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [22]:
# 予測の提出
submission_id = napi.upload_predictions(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv", model_id=model_id)

2021-10-09 18:04:37,741 INFO numerapi.base_api: uploading predictions...
